# 试用Python做教程内的SQL分析过程

## 04 实现数据过滤

In [628]:
import pandas as pd
import numpy as np
df = pd.read_excel('./file/employee.xls')

# 只返回刘备的信息
df.loc[df['emp_name'] == '刘备']

# 返回入职日期大于某一天的
df.loc[df['hire_date'] > '2018-01-01', ['emp_name','hire_date']]

# 返回月薪在某某之间的
df.loc[(df['salary'] >= 10000)&(df['salary'] <= 15000), ['emp_name','salary']]

# 查找姓名特定的员工
df.loc[df['emp_name'].isin(['刘备','张飞','关羽']),['emp_name', 'emp_id']]

# 空值判断
df.loc[df['manager'].isnull(),['emp_name','manager']]

# 女性且月薪超过10000的员工
df.loc[(df['sex'] =='女') & (df['salary'] >= 10000),['emp_name', 'salary']]

# OR运算符的转换
df.loc[(df['emp_name'] == '刘备') | (df['emp_name'] == '关羽') | (df['emp_name'] == '张飞'), ['emp_name']]

# 实现NOT运算
df.loc[-(df['emp_name'].isin(['刘备','张飞','关羽'])),['emp_id','emp_name']]

# 测试优先级
df.loc[((df['dept_id'] == 2) | (df['dept_id'] == 3)) & (-df['bonus'].isnull()), ['emp_name','emp_id','bonus']]

# 数据去重
# df = df.drop_duplicates(subset = ['sex'],keep='first',inplace=False)
# df['sex']

# 思考题: 查找 2018 年 1 月 1 日之后入职，月薪小于 5000，并且奖金小于 1000（包括没有奖金）的员工。
df.loc[(df['hire_date'] > '2018-01-01') & (df['salary'] < 5000) & ((df['bonus'] < 1000)|df['bonus'].isnull()), ['emp_name','hire_date','salary','bonus']]

,emp_name,hire_date,salary,bonus
20,黄权,2018-03-14,4200,NaN
21,糜竺,2018-03-27,4300,NaN
22,邓芝,2018-11-11,4000,NaN
23,简雍,2019-05-11,4800,NaN
24,孙乾,2018-10-09,4700,NaN


## 05 模糊查找

In [294]:
# df.loc[df.emp_name.str.startswith('赵'),['emp_id','emp_name','sex']]

# df.loc[df.email.str.contains('dengzh'),['emp_id','emp_name','email']]

df = pd.DataFrame(['TEST@shuguo.com','test@shuguo','.123@shuguo.com','test+email@shuguo.cn','me.me@ shuguo.com','123.test@shuguo-sanguo.org'],columns=['email'])
df
df.loc[df.email.str.contains('^[a-zA-Z0-9]+[a-zA-Z0-9._-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}$')]

,email
0,TEST@shuguo.com
5,123.test@shuguo-sanguo.org


## 06 排序

In [330]:
df = pd.read_excel('./file/employee.xls')
df.sort_index(ascending=False)
df.sort_index(axis=1)
df.loc[df['dept_id'] == 4,['emp_name','salary']].sort_values(by='salary')
df.loc[df['dept_id'] == 4,['emp_name','salary','hire_date']].sort_values(by=['salary','hire_date'],ascending=(False, True))

df.loc[df['dept_id'] == 2, ['emp_name', 'bonus']].sort_values(by='bonus', ascending=False)

,emp_name,bonus
3,诸葛亮,8000.0
4,黄忠,NaN
5,魏延,NaN


## 数值函数

In [380]:
import math
df = pd.read_excel('./file/employee.xls')
df.loc[df['bonus'].isnull()]

df = pd.DataFrame({'a':[4,5,6,7],'b':[10,20,30,40],'c':[100,50,-30,-50]})
df.abs()
math.ceil(2.4)
math.exp(3)
math.floor(2.1)
math.log(3)
math.log(2,3)
math.sqrt(4)

2.0

## 文本处理函数

In [579]:
import pandas as pd
df = pd.read_excel('./file/employee.xls')

df['emp_name']  = df['emp_name'].str[:-2]+"*"+df['emp_name'].str[-1]

df['email']  = df['email'].replace("@","#",regex=True)
df.loc[:,['emp_id','emp_name','email']].head(5)


,emp_id,emp_name,email
0,1,*备,liubei#shuguo.com
1,2,*羽,guanyu#shuguo.com
2,3,*飞,zhangfei#shuguo.com
3,4,诸*亮,zhugeliang#shuguo.com
4,5,*忠,huangzhong#shuguo.com


## 日期和时间

In [641]:
import pandas as pd
import numpy as np
import datetime
dt = pd.to_datetime(['12/02/2021',np.datetime64('2021-02-21'),datetime.datetime(2021,2,21)])
dt
ts = pd.Timestamp('2016-10-30 00:00:00', tz='Europe/Helsinki')
print(ts)

2016-10-30 00:00:00+03:00


## CASE语句

In [693]:
import pandas as pd
import numpy as np
df = pd.read_excel('./file/employee.xls')

def depar(n):
    switch = {
        1: '行政管理部',
        2: '人力资源部',
        3: '财务部',
        4: '研发部',
        5: '销售部',
        6: '保卫部',   
    }
    return switch.get(n,None)
df['department']  = df['dept_id'].apply(depar)

# 练习题：编写 SQL 语句查询员工信息，按照部门编号进行排序，并且确保同一个部门中的女性员工排在男性员工之前。
df[['emp_name','dept_id','sex']].sort_values(by=['dept_id','sex'], ascending=(True, True))

,emp_name,dept_id,sex
0,刘备,1,男
1,关羽,1,男
2,张飞,1,男
3,诸葛亮,2,男
4,黄忠,2,男
5,魏延,2,男
6,孙尚香,3,女
7,孙丫鬟,3,女
11,赵氏,4,女
8,赵云,4,男


## 汇总分析

In [775]:
import pandas as pd
import numpy as np
df = pd.read_excel('./file/employee.xls')

df['sex'].value_counts()

df.shape[0] - df['bonus'].count()



df['salary'].mean()

# df['salary'].drop_duplicates().mean()

# df['bonus'].mean()

# df['salary'].sum()

# df['hire_date'].max()

# df['hire_date'].min()

# df['salary'].var()

# df['salary'].std()

def depar(n):
    switch = {
        1: '行政管理部',
        2: '人力资源部',
        3: '财务部',
        4: '研发部',
        5: '销售部',
        6: '保卫部',   
    }
    return switch.get(n,None)
df['department']  = df['dept_id'].apply(depar)
df['department'].value_counts()

研发部      9
销售部      8
行政管理部    3
人力资源部    3
财务部      2
Name: department, dtype: int64

## 分组统计

In [805]:
import pandas as pd
import numpy as np
df = pd.read_excel('./file/employee.xls')
df.groupby('sex').size()
df.groupby(['dept_id','sex']).size()
df.groupby(['sex',df['salary'].mean()]).count().df['salary'].mean()

KeyError: 9832.0

## SQL替代


In [640]:
import pandas as pd
import numpy as np

df = pd.read_csv("./file/tips.csv")
df.head()

# SELECT

# SELECT total_bill, tip, smoker, time
# FROM df
# LIMIT 5;

df[['total_bill','tip','smoker','time']].head(5)

# SELECT *, tip/total_bill as tip_rate
# FROM df
# LIMIT 5;

df.assign(tip_rate=df['tip']/df['total_bill']).head(5)


# WHERE

# SELECT *
# FROM df
# WHERE time = 'Dinner'
# LIMIT 5;

df[df['time'] == 'Dinner'].head(5)

# -- tips of more than $5.00 at Dinner meals
# SELECT *
# FROM df
# WHERE time = 'Dinner' AND tip > 5.00;

df[(df['time'] == 'Dinner') & (df['tip'] > 5.00)]

# -- tips by parties of at least 5 diners OR bill total was more than $45
# SELECT *
# FROM tips
# WHERE size >= 5 OR total_bill > 45;

df[(df['size'] >= 5) | (df['total_bill'] > 45)]

frame = pd.DataFrame({"col1": ["A", "B", np.NaN, "C", "D"], "col2": ["F", np.NaN, "G", "H", "I"]})

# SELECT *
# FROM frame
# WHERE col2 IS NULL;

frame[frame['col2'].isna()]

# SELECT *
# FROM frame
# WHERE col1 IS NOT NULL;

# frame[frame['col1'].notna()]
frame[-frame['col1'].isna()]

# GROUP BY

# SELECT sex, count(*)
# FROM df
# GROUP BY sex;
# /*
# Female     87
# Male      157
# */

df.groupby('sex').size()
df.groupby('sex').count()

# SELECT day, AVG(tip), COUNT(*)
# FROM tips
# GROUP BY day;
# /*
# Fri   2.734737   19
# Sat   2.993103   87
# Sun   3.255132   76
# Thur  2.771452   62
# */

df.groupby('day').agg({'tip':np.mean, 'day':np.size})

# SELECT smoker, day, COUNT(*), AVG(tip)
# FROM tips
# GROUP BY smoker, day;
# /*
# smoker day
# No     Fri      4  2.812500
#        Sat     45  3.102889
#        Sun     57  3.167895
#        Thur    45  2.673778
# Yes    Fri     15  2.714000
#        Sat     42  2.875476
#        Sun     19  3.516842
#        Thur    17  3.030000
# */

df.groupby(['smoker','day']).agg({'tip':[np.size, np.mean]})

# JOIN

df1 = pd.DataFrame({"key": ["A", "B", "C", "D"], "value": np.random.randn(4)})
df2 = pd.DataFrame({"key": ["B", "D", "D", "E"], "value": np.random.randn(4)})

# SELECT *
# FROM df1
# INNER JOIN df2
#   ON df1.key = df2.key;

pd.merge(df1,df2,on='key')

# -- show all records from df1
# SELECT *
# FROM df1
# LEFT OUTER JOIN df2
#   ON df1.key = df2.key;

pd.merge(df1, df2, on='key', how='left')

# -- show all records from df2
# SELECT *
# FROM df1
# RIGHT OUTER JOIN df2
#   ON df1.key = df2.key;

pd.merge(df1,df2, on='key',how='right')

# -- show all records from both tables
# SELECT *
# FROM df1
# FULL OUTER JOIN df2
#   ON df1.key = df2.key;

pd.merge(df1,df2,on='key',how='outer')

df1 = pd.DataFrame({"city": ["Chicago", "San Francisco", "New York City"], "rank": range(1, 4)})
df2 = pd.DataFrame({"city": ["Chicago", "Boston", "Los Angeles"], "rank": [1, 4, 5]})

# SELECT city, rank
# FROM df1
# UNION ALL
# SELECT city, rank
# FROM df2;
# /*
#          city  rank
#       Chicago     1
# San Francisco     2
# New York City     3
#       Chicago     1
#        Boston     4
#   Los Angeles     5
# */

pd.concat([df1,df2])

# SELECT city, rank
# FROM df1
# UNION
# SELECT city, rank
# FROM df2;
# -- notice that there is only one Chicago record this time
# /*
#          city  rank
#       Chicago     1
# San Francisco     2
# New York City     3
#        Boston     4
#   Los Angeles     5
# */

pd.concat([df1,df2]).drop_duplicates()

# -- MySQL
# SELECT * FROM df
# ORDER BY tip DESC
# LIMIT 10 OFFSET 5;

df.nlargest(10+5, columns='tip').tail(10)


# -- Oracle's ROW_NUMBER() analytic function
# SELECT * FROM (
#   SELECT
#     t.*,
#     ROW_NUMBER() OVER(PARTITION BY day ORDER BY total_bill DESC) AS rn
#   FROM df t
# )
# WHERE rn < 3
# ORDER BY day, rn;

(df.assign(rn=df.sort_values(['total_bill'], ascending=False).groupby(['day']).cumcount()+1).query('rn<3').sort_values(['day','rn']))

,total_bill,tip,sex,smoker,day,time,size,rn
95,40.17,4.73,Male,Yes,Fri,Dinner,4,1
90,28.97,3.00,Male,Yes,Fri,Dinner,2,2
170,50.81,10.00,Male,Yes,Sat,Dinner,3,1
212,48.33,9.00,Male,No,Sat,Dinner,4,2
156,48.17,5.00,Male,No,Sun,Dinner,6,1
182,45.35,3.50,Male,Yes,Sun,Dinner,3,2
197,43.11,5.00,Female,Yes,Thur,Lunch,4,1
142,41.19,5.00,Male,No,Thur,Lunch,5,2
